# Simulated Classifcation

1. State assumptions
2. Formally define classification/regression problem
3. provide algorithm for solving problem (including choosing hyperparameters as appropriate)
4. sample data from a simulation setting inspired by your data (from both null and alternative as defined before)
5. compute accuracy
6. plot accuracy vs. sample size in simulation
7. apply method directly on real data
8. explain the degree to which you believe the result and why

## Step 1: State assumptions



## Step 2: Formally define classification/regression problem

## Step 3: Provide algorithm for solving problem (including choosing hyperparameters as appropriate)



In [1]:
# change working dir
path = "/Users/david/Desktop/CourseWork/TheArtOfDataScience/claritycontrol/code/scripts" # use your own path
import os
os.chdir(path)

### Histogram data preparation

> Skip this step if the data are already baked, just eat them!

1. Set suitable data value range to get histogram from the majority of the datasets.

In [2]:
import numpy as np
import clarity.resources as rs
import clarity as cl
import gc  # garbage memory collection :)

BINS=4
RANGE=(10.0,300.0)

# For testing, find the proper range.
# import matplotlib.pyplot as plt
# c = cl.Clarity("Cocaine178")
# hist, bin_edges = c.loadImg(info=False).getHistogram(bins=BINS,range=RANGE,density=True)
# width = 0.8*(bin_edges[1] - bin_edges[0])
# center = (bin_edges[:-1] + bin_edges[1:]) / 2
# plt.bar(center, hist, align='center', width=width)
# plt.show()

for token in rs.TOKENS:
    c = cl.Clarity(token)
    hist, bin_edges = c.loadImg(info=False).getHistogram(bins=BINS,range=RANGE,density=True)
    fname = "../data/hist/"+token+".hist"
    np.savetxt(fname,hist,delimiter=',')
    print fname,"saved."
    del c
    gc.collect()

Image Loaded: /Users/david/Desktop/CourseWork/TheArtOfDataScience/claritycontrol/code/data/./Cocaine174.img
../data/hist/Cocaine174.hist saved.
Image Loaded: /Users/david/Desktop/CourseWork/TheArtOfDataScience/claritycontrol/code/data/./Cocaine175.img
../data/hist/Cocaine175.hist saved.
Image Loaded: /Users/david/Desktop/CourseWork/TheArtOfDataScience/claritycontrol/code/data/./Cocaine178.img
../data/hist/Cocaine178.hist saved.
Image Loaded: /Users/david/Desktop/CourseWork/TheArtOfDataScience/claritycontrol/code/data/./Control181.img
../data/hist/Control181.hist saved.
Image Loaded: /Users/david/Desktop/CourseWork/TheArtOfDataScience/claritycontrol/code/data/./Control182.img
../data/hist/Control182.hist saved.
Image Loaded: /Users/david/Desktop/CourseWork/TheArtOfDataScience/claritycontrol/code/data/./Control189.img
../data/hist/Control189.hist saved.
Image Loaded: /Users/david/Desktop/CourseWork/TheArtOfDataScience/claritycontrol/code/data/./Control239.img
../data/hist/Control239.hist